In [1]:
import pandas as pd 
import sqlalchemy as sql
import alpaca_trade_api as tradeapi 
import os
from dotenv import load_dotenv
#import hvplot.pandas 
%matplotlib inline 

In [2]:
load_dotenv()

True

In [3]:
alpaca_api_key = os.getenv("ALPACA_API_KEY")
alpaca_secret_key = os.getenv("ALPACA_SECRET_KEY")
display(type(alpaca_api_key))
display(type(alpaca_secret_key))

str

str

In [4]:
#building the API, setting timeframe and daily prices  

alpaca=tradeapi.REST(
    alpaca_api_key,
    alpaca_secret_key,
    api_version = 'v2')

tickers = ['USO', 'TSLA', 'SPY']
start_date = pd.Timestamp("2010-06-29", tz="America/New_York").isoformat() #according to yahoo finance tsla issued shares on this date
end_date = pd.Timestamp("2021-10-15", tz="America/New_York").isoformat()

timeframe = "1D"

In [5]:
#creating pandas df with alpaca API

prices_df = alpaca.get_barset(
    tickers,
    timeframe,
    start=start_date,
    end=end_date,
).df


In [6]:
#creating closing prices df from larger prices_df 

closing_prices_df = pd.DataFrame()

for ticker in tickers:
    closing_prices_df[ticker] = prices_df[ticker]["close"]

closing_prices_df.index = closing_prices_df.index.date

closing_prices_df.index.name ='Date'
#closing_prices_df.columns(['date', 'USO', 'TSLA', 'SPY'])

closing_prices_df.head()

,USO,TSLA,SPY
Date,,,
2021-05-26,45.210,619.1900,419.04
2021-05-27,45.555,630.6000,419.69
2021-05-28,45.430,625.7100,420.02
2021-06-01,46.330,624.0300,419.63
2021-06-02,46.910,605.0001,420.38


In [7]:
# Working with SQL in order to not ping the Alpaca API too much  
#Creating SQL connection/engine

database_connection_string = 'sqlite:///'
engine = sql.create_engine(database_connection_string)
engine

Engine(sqlite:///)

In [8]:
#shoving closing_prices_df from Alpaca into a SQL Table 

closing_prices_df.to_sql('closing_prices', engine, index= True, if_exists='replace' )


In [13]:
engine.table_names()


<ipython-input-13-d72bc75a6b62>:1: SADeprecationWarning: The Engine.table_names() method is deprecated and will be removed in a future release.  Please refer to Inspector.get_table_names(). (deprecated since: 1.4)
  engine.table_names()


['closing_prices']

In [15]:
#Code to grab the data from the SQL table into a pandas df to analyze further 

closing_prices_sql_df = pd.read_sql_table ('closing_prices', con=engine, parse_dates=['Date'])
closing_prices_sql_df.head()

,Date,USO,TSLA,SPY
0,2021-05-26,45.210,619.1900,419.04
1,2021-05-27,45.555,630.6000,419.69
2,2021-05-28,45.430,625.7100,420.02
3,2021-06-01,46.330,624.0300,419.63
4,2021-06-02,46.910,605.0001,420.38


In [31]:
closing_prices_sql_df['USO_pct_change']= closing_prices_sql_df['USO'].pct_change().dropna()
closing_prices_sql_df['TSLA_pct_change']= closing_prices_sql_df['TSLA'].pct_change().dropna()
closing_prices_sql_df['SPY_pct_change']= closing_prices_sql_df['SPY'].pct_change().dropna()

closing_prices_sql_df.head()

,USO,TSLA,SPY,USO_pct_change,TSLA_pct_change,SPY_pct_change
Date,,,,,,
2021-05-26,45.210,619.1900,419.04,NaN,NaN,NaN
2021-05-27,45.555,630.6000,419.69,0.007631,0.018427,0.001551
2021-05-28,45.430,625.7100,420.02,-0.002744,-0.007755,0.000786
2021-06-01,46.330,624.0300,419.63,0.019811,-0.002685,-0.000929
2021-06-02,46.910,605.0001,420.38,0.012519,-0.030495,0.001787
